In [108]:
import pandas as pd

In [79]:
joined_df = pd.read_csv('C:/Users/APIN PC/OneDrive/Documents/DS/DE_Inter/etl_pipeline/data/Joined_data')

In [80]:
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1993 entries, 0 to 1992
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1993 non-null   object 
 1   release_year  1993 non-null   object 
 2   runtime       1654 non-null   object 
 3   genres        1993 non-null   object 
 4   rating        1977 non-null   float64
 5   description   1984 non-null   object 
 6   director      1000 non-null   object 
 7   votes_count   1977 non-null   float64
 8   total_sales   1334 non-null   object 
 9   type          1993 non-null   object 
 10  budget        408 non-null    float64
dtypes: float64(3), object(8)
memory usage: 171.4+ KB


####  `Convert runtime to integer.`
#### `Convert total sales to float`
#### `Drop other missing values`
#### `Convert release year to date (year)`
#### `Dropping director and budget column?`


In [ ]:
def transform_df() -> pd.DataFrame:
    """
    Transform joined dataframe 
    Returns:
        _type_: _description_
    """
    # Changing Object Types
    joined_df['runtime'] = joined_df['runtime'].astype(str).str.extract(r'(\d+)') # Extract only integer value.
    joined_df['runtime'] = pd.to_numeric(joined_df['runtime'], errors='coerce') # Change column to numeric
    joined_df['total_sales']= joined_df['total_sales'].astype(str).replace(r'\,', '', regex=True).astype(float)
    joined_df.loc[joined_df['release_year'] == 'PG', 'release_year'] = '1995' # Changing wrong release year to the right one.
    pd.to_datetime(joined_df['release_year'], format= '%Y').dt.year # Changing column to date type.
    # Treating NAs and Dropping Unwanted Columns
    joined_df = joined_df.dropna(subset=['votes_count', 'rating', 'description']) # Dropping unwanted missing values.
    joined_df = joined_df.drop(columns=['budget']) # Dropping budget column 
    
    return joined_df


<class 'pandas.core.frame.DataFrame'>
Index: 1973 entries, 0 to 1992
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1973 non-null   object 
 1   release_year  1973 non-null   object 
 2   runtime       1647 non-null   float64
 3   genres        1973 non-null   object 
 4   rating        1973 non-null   float64
 5   description   1973 non-null   object 
 6   director      1000 non-null   object 
 7   votes_count   1973 non-null   float64
 8   total_sales   1334 non-null   float64
 9   type          1973 non-null   object 
dtypes: float64(4), object(6)
memory usage: 169.6+ KB


In [129]:
joined_df.describe()
joined_df.isna().sum()

title             0
release_year      0
runtime         326
genres            0
rating            0
description       0
director        973
votes_count       0
total_sales     639
type              0
dtype: int64

I intentionally left missing values so as to show some peculiarities of the scraped data.
E.g Type column consists of -> movie, tvseries and tvshortseries, 
Only movie have record for runtime. Both tvseries and tvshortseries are null. 
It could mean that tvseries are still running or that the IMDb does not keep record of series runtime. 

In [130]:
joined_df.columns

Index(['title', 'release_year', 'runtime', 'genres', 'rating', 'description',
       'director', 'votes_count', 'total_sales', 'type'],
      dtype='object')